In [1]:
# Author : Mostafa.Hassani1983@gmail.com
import pandas as pd
from tqdm import tqdm_notebook
from scipy.spatial.distance import cosine
from sklearn.preprocessing import MinMaxScaler

In [1]:
DF_Sample = pd.read_csv("Write Your DataSet Address")
Columns = DF_Sample.columns
Scaler = MinMaxScaler()
DF_Sample[Columns] = Scaler.fit_transform(DF_Sample[Columns])

In [ ]:
# Fill Null Values With a Epcilon For Use In Next Part
DF_Sample = DF_Sample.fillna(-0.000001)

In [ ]:
%%time
# Search Similar Rows For Filling Null Values
N_Row = len(DF_Sample)
for Rows in tqdm_notebook(range(N_Row),desc="Rows : "):
    input_arr = list(DF_Sample.iloc[Rows])
    Similarity_List = DF_Sample.apply(lambda row: 1-cosine(row,input_arr),axis=1).sort_values(ascending=False)
#    Select columns containing value Zeroes
    Filter_Zero = (DF_Sample.iloc[[Rows]] == -0.000001).any()
    DF_Subset = DF_Sample.loc[:,Filter_Zero]
    Columns_DF_Sub = DF_Subset.columns
    for Col_Sub in Columns_DF_Sub:
        for Sim in range(len(Similarity_List)):
            if Rows!=Similarity_List.index[Sim] and 
                                        DF_Sample.at[Similarity_List.index[Sim],Col_Sub] != -0.000001 :
                if Similarity_List.values[Sim]>0.75 :
                    DF_Sample.at[Rows,Col_Sub] = DF_Sample.at[Similarity_List.index[Sim],Col_Sub]                
                elif Similarity_List.values[Sim]>0.5 :
                    DF_Temp = DF_Sample.iloc[(Similarity_List
                                              [(Similarity_List>0.5) & (Similarity_List<0.99999)].index)]
                    DF_Sample.at[Rows,Col_Sub] = DF_Temp[Col_Sub].mean()
                elif Similarity_List.values[Sim]>0 :
                    DF_Temp = DF_Sample.iloc[(Similarity_List
                                              [(Similarity_List>0) & (Similarity_List<0.49999)].index)]
                    DF_Sample.at[Rows,Col_Sub] = DF_Temp[Col_Sub].mean()
                else:
                    DF_Sample.at[Rows,Col_Sub] = DF_Sample[Col_Sub].mean()  # Mean, Median, mod, ....
                break
print(DF_Sample)